In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from transformers import ElectraForPreTraining, ElectraTokenizerFast
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
model = ElectraForPreTraining.from_pretrained('Seznam/small-e-czech')

In [ ]:
tokenizer = ElectraTokenizerFast.from_pretrained("Seznam/small-e-czech")

In [ ]:
from word_filter import WordFilterLayers, FilterBuilderWithTokenizer

In [ ]:
filter = WordFilterLayers()

In [ ]:
filter_builder = FilterBuilderWithTokenizer('stenozaznamy_psp_sample.pickle', tokenizer)

In [ ]:
nulles = filter_builder.get_nulls()
# for nul in sorted(nulles, key=(lambda x: nulles[x]), reverse=True):
#     print(f'{nul}:{nulles[nul]}')

In [ ]:
layer = WordFilterLayers()

In [ ]:
loss = nn.BCELoss()
optimizer = optim.Adam(layer.parameters(), lr=1e-4)

labels = {word: 1.0 * (nulles[word] > 0.9) for word in nulles}
counter = 0
for paragraph in filter_builder.paragraphs():
    counter += 1
    labels_t = []
    # for word in filter_builder.word_buffer(paragraph):
    #     encodings = tokenizer.encode(word)
    #     for encoding in encodings[1:-1]:
    #         if word in labels:
    #             labels_t.append((labels[word], encoding))
    #         else:
    #             labels_t.append((1.0, encoding))
    # print(tokenizer.encode(paragraph))
    # print(paragraph_encodings)
    #   paragraph_encodings = torch.tensor(paragraph_encodings)
    # counter = 0
    # for i in range(len(labels_t)):
    #     label_t = labels_t[counter] 
    #     while label_t[1] != paragraph_encodings[0,counter]:
    #         labels_t.insert(i, (0.0, paragraph_encodings[0, counter]))
    #         counter += 1
    #     else:
    #         counter += 1
    # print(pred.last_hidden_state)
    # labels_t.append((0.0, 1))
    # labels_t.insert(0, (0.0,)
    paragraph_encodings = tokenizer.encode(paragraph, return_tensors='pt')
    if len(paragraph_encodings[0]) > 512:
        paragraph_encodings = paragraph_encodings[:, 0:512]
    for encoding in paragraph_encodings[0]:
        if encoding.item() not in labels:
            labels_t.append(0.0)
        else:
            labels_t.append(labels[encoding.item()])
    pred = model.electra(paragraph_encodings)
    res = layer(pred.last_hidden_state)
    
    
    optimizer.zero_grad()
    loss_value = loss(res.squeeze(0).squeeze(1), torch.tensor(labels_t))
    loss_value.backward()
    optimizer.step()
    if counter % 10 == 0:
        print(f'{loss_value.item()}')
        paragraph = 'Válka je špatná a inflace také.'
        paragraph_encodings = tokenizer.encode(paragraph, return_tensors='pt')
        pred = model.electra(paragraph_encodings)
        # print(pred.last_hidden_state)
        res = layer(pred.last_hidden_state)
        fake_sentence_tokens = ['s'] + tokenizer.tokenize(paragraph) + ['k']
        for token in fake_sentence_tokens:
            print("{:>7s}".format(token), end="")
        print()

        for prediction in res.squeeze():
            print("{:7.1f}".format(prediction), end="")
        print()


In [ ]:
paragraph = 'Válka je špatná a inflace také.'
paragraph_encodings = tokenizer.encode(paragraph, return_tensors='pt')
pred = model.electra(paragraph_encodings)
# print(pred.last_hidden_state)
res = layer(pred.last_hidden_state)
fake_sentence_tokens = ['s'] + tokenizer.tokenize(paragraph) + ['k']
for token in fake_sentence_tokens:
    print("{:>7s}".format(token), end="")
print()

for prediction in res.squeeze():
    print("{:7.1f}".format(prediction), end="")
print()

In [ ]:
nulles

In [ ]:
from word_filter import WordFilter

In [77]:
filter = WordFilter()

In [78]:
loss = nn.BCELoss()
optimizer = optim.RMSprop(filter.parameters(), lr=1e-4)

labels = {word: 1.0 * (nulles[word] > 0.9) for word in nulles}
counter = 0
for paragraph in filter_builder.paragraphs():
    counter += 1
    labels_t = []
    paragraph_encodings = tokenizer.encode(paragraph, return_tensors='pt')
    if len(paragraph_encodings[0]) > 512:
        paragraph_encodings = paragraph_encodings[:, 0:512]
    for encoding in paragraph_encodings[0]:
        if encoding.item() not in labels:
            labels_t.append(0.0)
        else:
            labels_t.append(labels[encoding.item()])
    # pred = model.electra(paragraph_encodings)
    res = filter(paragraph_encodings, model.electra.embeddings.word_embeddings)
    
    
    optimizer.zero_grad()
    loss_value = loss(res.squeeze(0).squeeze(1), torch.tensor(labels_t))
    loss_value.backward()
    optimizer.step()
    if counter % 1000 == 0:
        print(f'{loss_value.item()}')
        
        paragraph = 'Válka je špatná a inflace také.'
        paragraph_encodings = tokenizer.encode(paragraph, return_tensors='pt')
        res = filter(paragraph_encodings, model.electra.embeddings.word_embeddings)
        # print(res)
        fake_sentence_tokens = ['s'] + tokenizer.tokenize(paragraph) + ['k']
        for token in fake_sentence_tokens:
            print("{:>7s}".format(token), end="")
        print()

        for prediction in res.squeeze():
            print("{:7.1f}".format(prediction), end="")
        print()


0.5996633172035217
      s  válka     je špatná      ainflace   také      .      k
    0.3    0.5    0.4    0.5    0.3    0.5    0.4    0.3    0.3
0.5020788908004761
      s  válka     je špatná      ainflace   také      .      k
    0.2    0.5    0.3    0.6    0.2    0.6    0.4    0.2    0.2
0.42128250002861023
      s  válka     je špatná      ainflace   také      .      k
    0.1    0.6    0.2    0.7    0.1    0.6    0.4    0.1    0.1
0.3836243450641632
      s  válka     je špatná      ainflace   také      .      k
    0.0    0.7    0.1    0.7    0.0    0.7    0.4    0.1    0.1
0.3565623164176941
      s  válka     je špatná      ainflace   také      .      k
    0.0    0.8    0.1    0.8    0.0    0.8    0.4    0.0    0.0
0.39007654786109924
      s  válka     je špatná      ainflace   také      .      k
    0.0    0.8    0.1    0.8    0.0    0.8    0.4    0.0    0.0
0.3752056062221527
      s  válka     je špatná      ainflace   také      .      k
    0.0    0.8    0.0    0.9    0

KeyboardInterrupt: 

In [ ]:
counter = 0
for word in nulles:
    if nulles[word] > 0.99:
        counter += 1
print(counter / len(nulles))

In [ ]:
nulles

In [79]:
torch.save(filter.state_dict(), 'models\\filter.model')

In [76]:
for paragraph in filter_builder.paragraphs():
    
    paragraph_encodings = tokenizer.encode(paragraph, return_tensors='pt')
    res = filter(paragraph_encodings, model.electra.embeddings.word_embeddings)
    # print(res)
    fake_sentence_tokens = ['s'] + tokenizer.tokenize(paragraph) + ['k']
    for token in fake_sentence_tokens:
        print("{:>7s}".format(token), end="")
    print()

    for prediction in res.squeeze():
        print("{:7.1f}".format(prediction), end="")
    print()
    break

      s děkuji      .  pěknédopoledne      , váženíkolegové      a    váž  ##ená   panípředsedkyně      .   jakopředseda    pod   ##vý ##boru    prodopravu   bych  chtěl krátce    oko ##ment ##ovat   tuto   norm    ##u      .    jak     už   bylo řečeno      ,   tato  norma   měla    býtprojedná   ##na     už      vminulémvolebním období      .   měla    být transp   ##on##ována     do      2      .  srpna     20   ##21      .     my   tady   mámezpoždění    rok      a    dva měsíce      a     ačevropská komiseprodlou ##žila   tuto  lhůtu     do      6      .      9      .     20   ##22      ,    tak    ani     to  nesti##hneme      a     už   bylozahájeno     to    san ##kční řízení   vůči  českérepublice      .      k
    0.0    0.2    0.0    1.0    1.0    0.0    1.0    1.0    0.0    1.0    1.0    0.2    0.8    0.0    0.0    0.8    0.8    1.0    1.0    0.1    1.0    0.0    1.0    0.9    1.0    1.0    1.0    0.8    1.0    0.4    0.0    0.1    0.1    0.6    0.9    0.0    0.9    1.0    